In [1]:
import pyreadr

def read_rds(file):
    df = pyreadr.read_r(file)[None]
    
    return df

In [2]:
viability = read_rds('../data/PRISM_klaeger_imputed_tidy.rds')
viability.head()

,depmap_id,drug,klaeger_conc,imputed_viability
0,ACH-000007,Abemaciclib,3.000000e-09,0.974677
1,ACH-000007,Abemaciclib,1.000000e-08,0.942180
2,ACH-000007,Abemaciclib,3.000000e-08,0.884299
3,ACH-000007,Abemaciclib,1.000000e-07,0.767666
4,ACH-000007,Abemaciclib,3.000000e-07,0.554302


In [3]:
kis = read_rds('../data/klaeger_full_tidy.rds')
kis['gene_name'] = 'inh_' + kis['gene_name']
kis.head()

,drug,gene_name,relative_intensity,concentration_M
0,Abemaciclib,inh_AAK1,1.000000,0.000000e+00
1,Abemaciclib,inh_AAK1,0.905556,3.000000e-09
2,Abemaciclib,inh_AAK1,0.791054,1.000000e-08
3,Abemaciclib,inh_AAK1,0.738292,3.000000e-08
4,Abemaciclib,inh_AAK1,0.639565,1.000000e-07


In [4]:
# pivot so genes are columns
kis = kis.pivot(index=['drug', 'concentration_M'], columns='gene_name', values='relative_intensity').reset_index()
kis.columns.name = None
kis.head()

,drug,concentration_M,inh_AAK1,inh_ABL1,inh_ABL2,inh_ACAD10,inh_ACAD11,inh_ACADVL,inh_ACOX1,inh_ACOX3,...,inh_ULK3,inh_UNC119,inh_VDAC3,inh_WEE1,inh_YARS,inh_YES1,inh_YTHDF3;YTHDF1,inh_YWHAG,inh_YWHAQ,inh_ZAK
0,AC-480,0.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,AC-480,1.000000e-08,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,AC-480,3.000000e-08,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,AC-480,1.000000e-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# create kis_viability

In [5]:
common_drugs = set(viability['drug']).intersection(set(kis['drug']))
common_conc = set(viability['klaeger_conc']).intersection(set(kis['concentration_M']))

In [6]:
kis_viability = kis[(kis['drug'].isin(common_drugs)) & (kis['concentration_M'].isin(common_conc))]
kis_viability = kis_viability.sort_values(['drug', 'concentration_M']).reset_index(drop=True)
kis_viability = kis_viability.merge(viability, left_on=['drug', 'concentration_M'], right_on=['drug', 'klaeger_conc'], how='inner')
kis_viability = kis_viability.drop(columns=['klaeger_conc'])
kis_viability = kis_viability.rename(columns={'imputed_viability': 'viability'})
kis_viability = kis_viability[['depmap_id', 'drug', 'concentration_M'] + [col for col in kis_viability.columns if col not in ['depmap_id', 'drug', 'concentration_M']]]
kis_viability.head()

,depmap_id,drug,concentration_M,inh_AAK1,inh_ABL1,inh_ABL2,inh_ACAD10,inh_ACAD11,inh_ACADVL,inh_ACOX1,...,inh_UNC119,inh_VDAC3,inh_WEE1,inh_YARS,inh_YES1,inh_YTHDF3;YTHDF1,inh_YWHAG,inh_YWHAQ,inh_ZAK,viability
0,ACH-000007,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
1,ACH-000008,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.088750
2,ACH-000011,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
3,ACH-000012,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.298190
4,ACH-000013,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.458047
